# Análise Nutricional de Alimentos com Machine Learning

## Configuração do Ambiente

In [15]:
# Verificar se estamos no Google Colab
import sys
import os

IN_COLAB = 'google.colab' in sys.modules
%rm -rf toolkit

if IN_COLAB:
    %rm -rf toolkit
    # Clonar repositório no Colab
    !git clone https://github.com/systentandobr/toolkit.git
    %cd toolkit/python/machinelearning-analysis-service

    # Instalar dependências
    !pip install -r requirements.txt
else:
    # Configuração local
    sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..')))

## Imports e Configurações

In [10]:
import json
import os

# Importações do projeto
from src.models.food.nutrition_service import create_food_nutrition_service
from src.exporters.nutrition_exporter import NutritionExporter

ModuleNotFoundError: No module named 'src.models.food.nutrition_service'

## Download de Imagens de Exemplo

Se estiver no Colab, vamos baixar algumas imagens de exemplo

In [ ]:
# Criar diretório para imagens
os.makedirs('resources/food_images', exist_ok=True)

if IN_COLAB:
    # Baixar imagens de exemplo
    !wget https://example.com/apple.jpg -O resources/food_images/apple.jpg
    !wget https://example.com/banana.jpg -O resources/food_images/banana.jpg
    !wget https://example.com/pizza.jpg -O resources/food_images/pizza.jpg
else:
    # Usar imagens locais
    resource_dir = os.path.join(os.path.dirname(__file__), 'resources', 'food_images')

# Listar imagens disponíveis
food_images = [
    'resources/food_images/apple.jpg',
    'resources/food_images/banana.jpg',
    'resources/food_images/pizza.jpg'
]

## Análise Nutricional

In [ ]:
# Criar serviço de análise nutricional
nutrition_service = create_food_nutrition_service()

# Realizar análise em lote
batch_analysis = nutrition_service.batch_analyze_foods(food_images)

# Gerar relatório nutricional
nutrition_report = nutrition_service.generate_nutrition_report(batch_analysis)

# Exibir relatório
print(json.dumps(nutrition_report, indent=2))

## Exportação de Resultados

In [ ]:
# Criar diretório de saída
os.makedirs('outputs', exist_ok=True)

# Exportar em diferentes formatos
export_formats = ['json', 'csv', 'excel']

for format in export_formats:
    output_path = f'outputs/nutrition_report.{format}'
    NutritionExporter.export(nutrition_report, output_path, format)
    print(f"Relatório exportado: {output_path}")

# No Colab, permitir download dos arquivos
if IN_COLAB:
    from google.colab import files
    for format in export_formats:
        files.download(f'outputs/nutrition_report.{format}')

## Visualização dos Resultados

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Converter dados nutricionais para DataFrame
nutrition_data = pd.DataFrame([
    {
        'Nutriente': 'Calorias',
        'Quantidade': nutrition_report['total_nutrition']['calories']
    },
    {
        'Nutriente': 'Proteínas',
        'Quantidade': nutrition_report['total_nutrition']['proteins']
    },
    {
        'Nutriente': 'Carboidratos',
        'Quantidade': nutrition_report['total_nutrition']['carbohydrates']
    },
    {
        'Nutriente': 'Gorduras',
        'Quantidade': nutrition_report['total_nutrition']['fats']
    }
])

# Gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(nutrition_data['Nutriente'], nutrition_data['Quantidade'])
plt.title('Resumo Nutricional')
plt.xlabel('Nutrientes')
plt.ylabel('Quantidade')
plt.tight_layout()
plt.savefig('outputs/nutrition_summary.png')
plt.show()

# No Colab, fazer download do gráfico
if IN_COLAB:
    files.download('outputs/nutrition_summary.png')